Bitte Antworte in Deutsch, gebe mit einen Python Code aus.
Verwende dabei bitte die Frameworks keras und sklearn
um folgendes zu tun.

Einlesen eines Pandas Datenframes mit vielen spalten und Werten, wir brauchen aus diesem das Lieferdatum in PlannedDate, Kunde in PartnerName, Bestellmenge Karton in Pick Karton, Bestellmenge Stangen in Picks Stangen, Bestellmenge Paletten in Picks Paletten. Der Datensatz reicht von 04.2021 bis heute, nun möchte ich einen Forecast, welcher mir für die nächsten 5 Werktage die wahrscheinlichen Bestellmengen pro Tag und Kunde ausgibt. Bitte beachte das die Bestellmengen, oder Kunden auch Nullwerte enthalten können


In [8]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [9]:
df = pq.read_table('df.parquet.gzip').to_pandas()

In [11]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM



# Auswahl der benötigten Spalten
df = df[["PlannedDate", "PartnerName", "Picks Karton", "Picks Stangen", "Picks Paletten"]]

# Umwandlung des Lieferdatums in Datetime-Format
df["PlannedDate"] = pd.to_datetime(df["PlannedDate"])

# Gruppieren der Daten nach Kunden und Lieferdatum
df = df.groupby(["PartnerName", "PlannedDate"]).sum().reset_index()

# Iterieren über alle Kunden
for partner in df["PartnerName"].unique():
    # Auswahl der Daten des aktuellen Kunden
    df_partner = df[df["PartnerName"] == partner]
    df_partner = df_partner.set_index("PlannedDate")

    # Skalieren der Daten
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_partner["Picks Karton"] = scaler.fit_transform(df_partner[["Picks Karton"]])

    # Erstellen des TimeseriesGenerators
    n_input = 5 #min(5, len(df_partner))

    n_features = 1
    generator = TimeseriesGenerator(df_partner["Picks Karton"], df_partner["Picks Karton"], length=n_input, batch_size=1)

    # Erstellen und Trainieren des LSTM-Modells
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit_generator(generator, epochs=5)

    # Vorhersagen für die nächsten 5 Werktage
    predictions = model.predict_generator(generator)
    predictions = scaler.inverse_transform(predictions)
    print(f"Forecast für den Kunden {partner}: {predictions}")


Epoch 1/5


/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_37880/2904101.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, epochs=5)


143/143 [==============================] - 1s 2ms/step - loss: 0.0364
Epoch 2/5
143/143 [==============================] - 0s 2ms/step - loss: 0.0309
Epoch 3/5
143/143 [==============================] - 0s 2ms/step - loss: 0.0270
Epoch 4/5
143/143 [==============================] - 0s 2ms/step - loss: 0.0265
Epoch 5/5
143/143 [==============================] - 0s 2ms/step - loss: 0.0286
Forecast für den Kunden 0: [[  53.88611  ]
 [  15.63442  ]
 [  21.427937 ]
 [  21.769356 ]
 [  30.247406 ]
 [  35.544846 ]
 [  36.290966 ]
 [  36.615685 ]
 [  42.09138  ]
 [  28.392687 ]
 [  28.587982 ]
 [  30.113474 ]
 [  22.825083 ]
 [  22.515305 ]
 [  20.41173  ]
 [  19.821648 ]
 [  21.221252 ]
 [  20.72886  ]
 [ 230.4766   ]
 [ 349.03717  ]
 [ 496.5815   ]
 [ 787.39984  ]
 [ 976.62274  ]
 [ 984.6959   ]
 [1057.9712   ]
 [1163.4651   ]
 [1050.228    ]
 [1097.2645   ]
 [ 922.34796  ]
 [1172.6604   ]
 [1091.9841   ]
 [1103.1965   ]
 [1098.569    ]
 [1257.4172   ]
 [ 794.08203  ]
 [ 836.6511   ]
 [ 992.

/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_37880/2904101.py:43: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(generator)


Epoch 1/5


...

/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_37880/2904101.py:43: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(generator)
/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_37880/2904101.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, epochs=5)


26/26 [==============================] - 1s 2ms/step - loss: 0.0385
Epoch 2/5
26/26 [==============================] - 0s 2ms/step - loss: 0.0384
Epoch 3/5
26/26 [==============================] - 0s 3ms/step - loss: 0.0383
Epoch 4/5
26/26 [==============================] - 0s 2ms/step - loss: 0.0377
Epoch 5/5
26/26 [==============================] - 0s 2ms/step - loss: 0.0376


/var/folders/0j/czkb_cmx12d_xchj2bf8bptc0000gn/T/ipykernel_37880/2904101.py:43: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(generator)


Forecast für den Kunden ALDI GmbH & Co. KG: [[ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [ 0.2839064 ]
 [-0.21620457]
 [-0.11788662]
 [-0.03806851]
 [ 0.02949389]]


ValueError: `start_index+length=5 > end_index=4` is disallowed, as no part of the sequence would be left to be used as current step.